In [3]:
import gradio as gr
import pickle
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load the trained model for a specific city
def load_model(city_name):
    # Convert city name to lowercase
    city_name_lower = city_name.lower()
    model_file_path = f"{city_name_lower}_model.pkl"
    with open(model_file_path, 'rb') as f:
        model = pickle.load(f)
    return model

# Your forecast_temperature function
def forecast_temperature(city_name, show_average_temperature):
    # Convert city name to lowercase
    city_name_lower = city_name.lower()
    
    # Load the model for the specified city
    model = load_model(city_name_lower)
    
    # Assuming your model has a method to get the forecast for the next 12 hours
    forecast = model.get_forecast(steps=12)  # Replace with your actual method
    
    # Convert Series to DataFrame
    forecast_table = forecast.predicted_mean.reset_index().rename(columns={'index': 'timestamp', 0: 'temperature'})
    
    # Calculate average temperature
    average_temperature = forecast_table['predicted_mean'].mean()
    max_temperature = forecast_table['predicted_mean'].max()
    min_temperature = forecast_table['predicted_mean'].min()

    # Create the output strings
    output_string = f"Forecasted Temperature for {city_name_lower} °C:\nForecasted Table:\n{forecast_table.to_string(index=False)}"

    avg_output_string = f"Average Temperature: {average_temperature:.2f} °C"
    max_output_string = f"Maximum Temperature: {max_temperature:.2f} °C"
    min_output_string = f"Minimum Temperature: {min_temperature:.2f} °C"

    return output_string, avg_output_string, max_output_string, min_output_string

# Create Gradio interface with radio buttons and checkbox
city_radio = gr.Radio(
    choices=["Agartala", "Amritsar", "Bangalore", "Chennai", "Delhi", "Faridabad", "Jabalpur", "Mumbai", "Pune", "Shimla"],
    type="value",
    label="Select City"
)

iface = gr.Interface(
    fn=forecast_temperature,
    inputs=city_radio,
    outputs=["text", "text", "text", "text"],
    title="Weather Forecast",
    live=True,
    theme="compact",  # Change theme to "compact" for a cleaner appearance
    css="""
        body {
            font-family: Arial, sans-serif;
        }
        .output {
            margin-top: 20px;
            text-align: center;
        }
    """
)

# Launch Gradio interface
iface.launch()


/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/gradio/blocks.py:530: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-65d4d63a-3f79f3f469a96ec071224926;2055116f-6f4d-43c5-a7bb-663cef9a2901)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/gradio/utils.py:854: UserWarning: Expected 2 arguments for function <function forecast_temperature at 0x7f7f917d5940>, received 1.
  warnings.warn(
/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/gradio/utils.py:858: UserWarning: Expected at least 2 arguments for function <function forecast_temperature at 0x7f7f917d5940>, received 1.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/gradio/helpers.py:821: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
